Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

## Logistic regression

In [5]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 10000
beta = 1e-3

graph = tf.Graph()
with graph.as_default():

  # Input data.
  # Load the training, validation and test data into constants that are
  # attached to the graph.
  tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
  tf_train_labels = tf.constant(train_labels[:train_subset])
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  # These are the parameters that we are going to be training. The weight
  # matrix will be initialized using random values following a (truncated)
  # normal distribution. The biases get initialized to zero.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  # We multiply the inputs with the weight matrix, and add biases. We compute
  # the softmax and cross-entropy (it's one operation in TensorFlow, because
  # it's very common, and it can be optimized). We take the average of this
  # cross-entropy across all training examples: that's our loss.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) \
    + beta * tf.nn.l2_loss(weights)
  
  # Optimizer.
  # We are going to find the minimum of this loss using gradient descent.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  # These are not part of training, but merely here so that we can report
  # accuracy figures as we train.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [6]:
num_steps = 801

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

with tf.Session(graph=graph) as session:
  # This is a one-time operation which ensures the parameters get initialized as
  # we described in the graph: random weights for the matrix, zeros for the
  # biases. 
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    # Run the computations. We tell .run() that we want to run the optimizer,
    # and get the loss value and the training predictions returned as numpy
    # arrays.
    _, l, predictions = session.run([optimizer, loss, train_prediction])
    if (step % 100 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.1f%%' % accuracy(
        predictions, train_labels[:train_subset, :]))
      # Calling .eval() on valid_prediction is basically like calling run(), but
      # just to get that one numpy array. Note that it recomputes all its graph
      # dependencies.
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 18.977570
Training accuracy: 15.3%
Validation accuracy: 19.5%
Loss at step 100: 4.882923
Training accuracy: 71.7%
Validation accuracy: 71.6%
Loss at step 200: 4.031670
Training accuracy: 74.8%
Validation accuracy: 74.2%
Loss at step 300: 3.453768
Training accuracy: 76.2%
Validation accuracy: 75.1%
Loss at step 400: 3.009914
Training accuracy: 77.4%
Validation accuracy: 75.8%
Loss at step 500: 2.653330
Training accuracy: 78.3%
Validation accuracy: 76.2%
Loss at step 600: 2.359870
Training accuracy: 79.0%
Validation accuracy: 76.6%
Loss at step 700: 2.114534
Training accuracy: 79.8%
Validation accuracy: 77.0%
Loss at step 800: 1.907205
Training accuracy: 80.5%
Validation accuracy: 77.2%
Test accuracy: 84.5%


## Neural network

In [11]:
batch_size = 128
beta = 1e-3

n_hidden_1 = 1024 # 1st layer number of features
n_hidden_2 = 1024 # 2nd layer number of features

graph = tf.Graph()

def pred_fn(x,  weights, biases):
  layer_1 = tf.matmul(x, weights['h1']) + biases['h1']
  layer_1 = tf.nn.relu(layer_1)
  layer_2 = tf.matmul(layer_1, weights['h2']) + biases['h2']
  layer_2 = tf.nn.relu(layer_2)
  output = tf.matmul(layer_2,  weights['out']) + biases['out']
  return output

with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = {
      'h1': tf.Variable(tf.truncated_normal([image_size * image_size, n_hidden_1])),
      'h2': tf.Variable(tf.truncated_normal([n_hidden_1, n_hidden_2])),
      'out': tf.Variable(tf.truncated_normal([n_hidden_2, num_labels]))
  }

  biases = {
      'h1' : tf.Variable(tf.zeros([n_hidden_1])),
      'h2' : tf.Variable(tf.zeros([n_hidden_2])),
      'out' : tf.Variable(tf.zeros([num_labels]))
  }
  
  # Training computation.
  logits = pred_fn(tf_train_dataset, weights, biases)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) \
    + beta * (tf.nn.l2_loss(weights['h1']) + tf.nn.l2_loss(weights['h2']) + tf.nn.l2_loss(weights['out']))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(pred_fn(tf_valid_dataset, weights, biases))
  test_prediction = tf.nn.softmax(pred_fn(tf_test_dataset, weights, biases))

In [12]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 1175.784180
Minibatch accuracy: 0.0%
Validation accuracy: 23.3%
Minibatch loss at step 500: 518.605591
Minibatch accuracy: 75.0%
Validation accuracy: 65.7%
Minibatch loss at step 1000: 619.171997
Minibatch accuracy: 56.2%
Validation accuracy: 61.3%
Minibatch loss at step 1500: 178.933899
Minibatch accuracy: 62.5%
Validation accuracy: 68.5%
Minibatch loss at step 2000: 106.906883
Minibatch accuracy: 75.0%
Validation accuracy: 74.1%
Minibatch loss at step 2500: 65.754585
Minibatch accuracy: 50.0%
Validation accuracy: 77.1%
Minibatch loss at step 3000: 39.438454
Minibatch accuracy: 87.5%
Validation accuracy: 79.8%
Test accuracy: 87.2%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [ ]:
batch_size = 128
beta = 1e-3

n_hidden_1 = 1024 # 1st layer number of features
n_hidden_2 = 1024 # 2nd layer number of features

keep_prob=0.8

graph = tf.Graph()

def pred_fn(x,  weights, biases, dropout=False):
  layer_1 = tf.nn.relu(tf.matmul(x, weights['h1']) + biases['h1'])
  if dropout:
    layer_1 = tf.nn.dropout(layer_1, keep_prob)
  output = tf.matmul(layer_1,  weights['out']) + biases['out']
#   layer_2 = tf.matmul(layer_1, weights['h2']) + biases['h2']
#   layer_2 = tf.nn.relu(layer_2)
#   if dropout:
#     layer_2 = tf.nn.dropout(layer_2, keep_prob)
#   output = tf.matmul(layer_2,  weights['out']) + biases['out']
  return output

with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  
#   keep_prob = tf.placeholder(tf.float32)

  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = {
      'h1': tf.Variable(tf.truncated_normal([image_size * image_size, n_hidden_1])),
#       'h2': tf.Variable(tf.truncated_normal([n_hidden_1, n_hidden_2])),
      'out': tf.Variable(tf.truncated_normal([n_hidden_2, num_labels]))
  }

  biases = {
      'h1' : tf.Variable(tf.zeros([n_hidden_1])),
#       'h2' : tf.Variable(tf.zeros([n_hidden_2])),
      'out' : tf.Variable(tf.zeros([num_labels]))
  }
  
  # Training computation.
  logits = pred_fn(tf_train_dataset, weights, biases, dropout=True)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) 
#     + beta * (tf.nn.l2_loss(weights['h1']) + tf.nn.l2_loss(weights['h2']) + tf.nn.l2_loss(weights['out']))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(pred_fn(tf_valid_dataset, weights, biases))
  test_prediction = tf.nn.softmax(pred_fn(tf_test_dataset, weights, biases))

In [ ]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 430.662659
Minibatch accuracy: 11.7%
Validation accuracy: 32.2%
Minibatch loss at step 500: 24.426540
Minibatch accuracy: 69.5%
Validation accuracy: 79.2%
Minibatch loss at step 1000: 8.635721
Minibatch accuracy: 78.9%
Validation accuracy: 79.7%
Minibatch loss at step 1500: 10.716989
Minibatch accuracy: 77.3%
Validation accuracy: 80.0%
Minibatch loss at step 2000: 5.179000
Minibatch accuracy: 80.5%
Validation accuracy: 80.3%
Minibatch loss at step 2500: 15.135192
Minibatch accuracy: 78.9%
Validation accuracy: 80.9%
Minibatch loss at step 3000: 4.490461
Minibatch accuracy: 76.6%
Validation accuracy: 81.8%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---
